### 🛠️ 1. Initialize notebook variables

Configures everything that's needed for deployment.

**Modify entries under _1) User-defined parameters_**. The APIs are pre-configured for the valet key pattern implementation.

In [ ]:
import utils
from apimtypes import *

# 1) User-defined parameters (change these as needed)
rg_location = 'eastus2'
index       = 1
deployment  = INFRASTRUCTURE.SIMPLE_APIM  # This sample works with all infrastructures
tags        = ['secure-blob-access', 'valet-key', 'storage', 'jwt', 'authz']
api_prefix  = 'blob-'  # OPTIONAL: ENTER A PREFIX FOR THE APIS TO REDUCE COLLISION POTENTIAL WITH OTHER SAMPLES

# 2) Service-defined parameters (please do not change these)
rg_name                 = utils.get_infra_rg_name(deployment, index)
sample_folder           = "secure-blob-access"
nb_helper               = utils.NotebookHelper(sample_folder, rg_name, rg_location, deployment, [INFRASTRUCTURE.AFD_APIM_PE, INFRASTRUCTURE.APIM_ACA, INFRASTRUCTURE.SIMPLE_APIM], True)
jwt_key_name            = nb_helper.jwt_key_name
jwt_key_value_bytes_b64 = nb_helper.jwt_key_value_bytes_b64

# 3) Set up the named values
nvs: List[NamedValue] = [
    NamedValue(jwt_key_name, jwt_key_value_bytes_b64, True),
    NamedValue('HRMemberRoleId', Role.HR_MEMBER)
]

# 4) Set up the policy fragments
pf_authx_hr_member_xml = utils.read_policy_xml('pf-authx-hr-member.xml', {
    'jwt_signing_key': jwt_key_name,
    'hr_member_role_id': 'HRMemberRoleId'
}, sample_folder)

pf_create_sas_token_xml = utils.read_policy_xml('pf-create-sas-token.xml', sample_name = sample_folder)
pf_check_blob_existence_via_mi = utils.read_policy_xml('pf-check-blob-existence-via-managed-identity.xml', sample_name = sample_folder)

pfs: List[PolicyFragment] = [
    PolicyFragment('AuthX-HR-Member', pf_authx_hr_member_xml, 'Authenticates and authorizes users with HR Member role.'),
    PolicyFragment('Create-Sas-Token', pf_create_sas_token_xml, 'Creates a SAS token to use with access to a blob.'),
    PolicyFragment('Check-Blob-Existence-via-Managed-Identity', pf_check_blob_existence_via_mi, 'Checks whether the specified blob exists at the blobUrl. A boolean value for blobExists will be available afterwards.')
]

# 5) Define blob storage parameters
container_name = 'hr-assets'
file_name = 'hr.txt'

# Read the policy XML without modifications - it already uses correct APIM named value format
pol_blob_get = utils.read_and_modify_policy_xml('blob-get-operation.xml', {
    'container_name': container_name
}, sample_folder)                                       

# Define template parameters for the blob name
blob_template_parameters = [
    {
        "name": "blob-name",
        "description": "The name of the blob to access",
        "type": "string",
        "required": True
    }
]

# 6) Define the APIs and their operations and policies

# Secure Blob Access API
blob_get = GET_APIOperation2('GET', 'GET', '/{blob-name}', 'Gets the blob access valet key info', pol_blob_get, templateParameters = blob_template_parameters)
                             
secure_blob_api = API(name = 'secure-blob-access', displayName = 'Secure Blob Access API', path = f'/{api_prefix}secure-files', 
    description = 'API for secure access to blob storage using the valet key pattern', operations = [blob_get], tags = tags)

# APIs Array
apis: List[API] = [secure_blob_api]

utils.print_ok('Notebook initialized')

### 🚀 2. Create deployment using Bicep

Creates the bicep deployment into the previously-specified resource group. A bicep parameters file will be created prior to execution.

In [ ]:
import utils

# 1) Define the Bicep parameters with serialized APIs
bicep_parameters = {
    'apis': {'value': [api.to_dict() for api in apis]},
    'namedValues': {'value': [nv.to_dict() for nv in nvs]},
    'policyFragments': {'value': [pf.to_dict() for pf in pfs]},
    'containerName': {'value': container_name},
    'blobName': {'value': file_name}
}

# 2) Deploy the bicep template
output = nb_helper.deploy_bicep(bicep_parameters)

if output.json_data:
    apim_name            = output.get('apimServiceName', 'APIM Service Name')
    apim_gateway_url     = output.get('apimResourceGatewayURL', 'APIM API Gateway URL')
    storage_account_name = output.get('storageAccountName', 'Storage Account Name')
    storage_endpoint     = output.get('storageAccountEndpoint', 'Storage Endpoint')
    container_name       = output.get('blobContainerName', 'Blob Container Name')
    apim_apis            = output.getJson('apiOutputs', 'APIs')

utils.print_ok('Deployment completed')

### ✅ 3. Verify APIM Managed Identity Permissions

Before testing secure blob access, we need to ensure that APIM's managed identity has the correct permissions to read from the storage account. This check helps avoid confusion if role assignment propagation is still in progress. The deployment script attempts to assign the **Storage Blob Data Reader** role to APIM's managed identity, but Azure role assignments can take several minutes to propagate fully across all services.

In [ ]:
import utils

# Use the improved permission check utility function
utils.print_message('Verifying APIM Managed Identity Permissions', blank_above = True)

# Run the permission check with automatic retry and clear user feedback
permissions_ready = utils.wait_for_apim_blob_permissions(
    apim_name = apim_name,
    storage_account_name = storage_account_name,
    resource_group_name = rg_name,
    max_wait_minutes = 5  # Allow up to 5 minutes for role propagation
)

if permissions_ready:
    utils.print_ok('APIM permissions verified successfully')
else:
    utils.print_warning('Permission verification incomplete - you may encounter 503/403 errors during testing')
    print("💡 If you see 503 errors in the next step, wait a few minutes and try again.")

### ✅ 4. Test the Secure Blob Access with Authentication

Test the secure blob access API to verify both the authentication/authorization and valet key pattern implementation. We'll:
1. Create JWT tokens for authorized and unauthorized users
2. Test API access with valid authentication
3. Test access denial for unauthorized users  
4. Verify direct blob access using the valet key pattern

The sample file was automatically created during the infrastructure deployment using a Bicep deployment script.

❗️ If the infrastructure shields APIM and requires a different ingress (e.g. Azure Front Door), the request to the APIM gateway URL will fail by design. Obtain the Front Door endpoint hostname and try that instead.

🔍 **Note about 503/403 errors**: If you see Service Unavailable (503) or Forbidden (403) errors when accessing blobs through APIM, this is likely due to role assignment propagation delays. The permission check above helps identify this scenario.

In [ ]:
import utils
from apimrequests import ApimRequests
from apimtesting import ApimTesting
import requests
import json
from users import UserHelper
from authfactory import AuthFactory

def handleResponse(response):
    if isinstance(response, str):
        try:
            access_info = json.loads(response)
            sas_url = access_info.get('sas_url', 'N/A')

            if sas_url == 'N/A':
                return response

            print(f"\nSecure Blob URL: {sas_url}")
            print(f"Expires At: {access_info.get('expire_at', 'N/A')}")            

            # Test direct access to the blob using the provided credentials
            utils.print_message("🧪 Testing direct blob access...", blank_above = True)
            
            try:
                blob_response = requests.get(access_info['sas_url'])
                if blob_response.status_code == 200:
                    utils.print_info("Direct blob access successful!")
                    content_preview = blob_response.text[:200] + "..." if len(blob_response.text) > 200 else blob_response.text
                    utils.print_val(f"Content preview:", content_preview.strip(), True)
                    return content_preview.strip()
                else:
                    utils.print_error(f"Direct blob access failed: {blob_response.status_code}")
                    return blob_response.status_code
            except Exception as e:
                utils.print_error(f"Error accessing blob directly: {str(e)}")
        except (json.JSONDecodeError, AttributeError):
            utils.print_error("Failed to parse JSON response or response is not in expected format.")
    return response

tests = ApimTesting("Secure Blob Access Sample Tests", sample_folder, deployment)

api_subscription_key = apim_apis[0]['subscriptionPrimaryKey']

# Preflight: Check if the infrastructure architecture deployment uses Azure Front Door
endpoint_url = utils.test_url_preflight_check(deployment, rg_name, apim_gateway_url)

# 1) Test with authorized user (has blob access role)
utils.print_message('1) Testing with Authorized User', blank_above = True)

# Create a JWT with the HR Member role (blob access)
encoded_jwt_token_hr_member = AuthFactory.create_symmetric_jwt_token_for_user(UserHelper.get_user_by_role(Role.HR_MEMBER), nb_helper.jwt_key_value)
print(f'\nJWT token for HR Member:\n{encoded_jwt_token_hr_member}')  # this value is used to call the APIs via APIM

# Set up APIM requests with authorized JWT
reqsApimAuthorized = ApimRequests(endpoint_url, api_subscription_key)
reqsApimAuthorized.headers['Authorization'] = f'Bearer {encoded_jwt_token_hr_member}'

# Test sample file access
print(f"\n🔒 Getting secure access for {file_name} with authorized user...")
response = reqsApimAuthorized.singleGet(f'/{api_prefix}secure-files/{file_name}', msg = f'Requesting secure access for {file_name} (authorized)')
output = handleResponse(response)
tests.verify(output, 'This is an HR document.')

# 2) Test with unauthorized user (has blob access role)
utils.print_message('2) Testing with Unauthorized User', blank_above = True)

encoded_jwt_token_no_role = AuthFactory.create_symmetric_jwt_token_for_user(UserHelper.get_user_by_role(Role.NONE), nb_helper.jwt_key_value)
print(f'\nJWT token for user with no role:\n{encoded_jwt_token_no_role}')  # this value is used to call the APIs via APIM

# Set up APIM requests with unauthorized JWT
reqsApimAuthorized = ApimRequests(endpoint_url, api_subscription_key)
reqsApimAuthorized.headers['Authorization'] = f'Bearer {encoded_jwt_token_no_role}'

# Test sample file access
print(f"\n🔒 Attempting to obtain secure access for {file_name} with unauthorized user (expect 401/403)...")
response = reqsApimAuthorized.singleGet(f'/{api_prefix}secure-files/{file_name}', msg = f'Requesting secure access for {file_name} (authorized)')
output = handleResponse(response)
tests.verify(json.loads(output)['statusCode'], 401)

tests.print_summary()

utils.print_ok('All done!')
